In [ ]:
# some_file.py
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/home/orco/Dokumente/PTB/MRS/MRSproc')

In [ ]:
#from mrsproc.preprocess import *
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import suspect
import multiprocessing as mp
plt.rcParams['figure.figsize'] = [10, 7]

In [ ]:
os.getcwd()

In [ ]:
mm_dats = glob.glob('../sub-*/mrs/sub-*_nuc-1H_loc-*_spec-ad-special_TI1_765_TI2_35*.dat')

In [ ]:
mm_dats

In [ ]:
s1 = list()
s1.append(mm_dats.pop(-3))


In [ ]:
s1.append(mm_dats.pop(-3))
s1

In [ ]:
s11 = [suspect.io.load_twix(i) for i in s1]
s111 = np.append(s11[0], s11[1], axis=0)
s111 = s11[0].inherit(s111)

In [ ]:
mm_dats += ['../sub-001/mrs/sub-001_nuc-1H_loc-pcc_spec-ad-special_TI1_765_TI2_35.dat']
water = [i.replace('ad-special_TI1_765_TI2_35.dat','rm-special_water_wOVS.dat') for i in mm_dats]

In [ ]:
water

In [ ]:
mm_dats.pop(3)
water.pop(3)

In [ ]:
[os.path.isfile(i) for i in water]

In [ ]:
[os.path.isfile(i) for i in mm_dats]

In [ ]:
%%time
mms = [suspect.io.load_twix(i) for i in mm_dats]


In [ ]:
waters = [suspect.io.load_twix(i) for i in water]

In [ ]:
plt.plot(mms[0][0,0].frequency_axis_ppm(), mms[0][0,0].spectrum().real)
plt.plot(mms[0][1,0].frequency_axis_ppm(), mms[0][1,0].spectrum().real)
plt.xlim([5,0])

In [ ]:
def _avg_everytwo(arr):
    return 0.5*(arr[0::2,:, :] + arr[1::2,:, :])

In [ ]:
%%time
avg_everytwo_data = [_avg_everytwo(i) for i in mms]
avg_everytwo_water = [_avg_everytwo(i) for i in waters]

In [ ]:
plt.plot(avg_everytwo_data[0][0,0].frequency_axis_ppm(), avg_everytwo_data[0][0,0].spectrum().real)
plt.plot(avg_everytwo_data[0][1,0].frequency_axis_ppm(), avg_everytwo_data[0][1,0].spectrum().real)
plt.xlim([5,0])

In [ ]:
del(mms)
del(waters)

In [ ]:
#Rodgers, C. T., & Robson, M. D. (2010). Receive array magnetic resonance spectroscopy: Whitened singular value decomposition (WSVD) gives optimal Bayesian solution. Magnetic Resonance in Medicine, 63(4), 881–891
def coil_combine(data, wref, noise_points=None):
    if not noise_points:
        noise_points = int(data.shape[-1]/8)
    noise = data[:,:,-noise_points:]
    noise = np.moveaxis(noise, -2, 0).reshape((32, -1))
    white_data = suspect.processing.channel_combination.whiten(data, noise)
    white_wref = suspect.processing.channel_combination.whiten(wref, noise)
    noise = white_data[:, :, -noise_points:]
    channel_weights = suspect.processing.channel_combination.svd_weighting(np.mean(white_wref, axis=0))
    cc_data = suspect.processing.channel_combination.combine_channels(white_data, channel_weights)
    cc_wref = suspect.processing.channel_combination.combine_channels(white_wref, channel_weights)
    return cc_data, cc_wref

In [ ]:
len(avg_everytwo_data)

In [ ]:
len(avg_everytwo_water)

In [ ]:
%%time
cc_data = list()
cc_wref = list()
for n in range(len(avg_everytwo_data)):
    print(n)
    d,w = coil_combine(avg_everytwo_data[n],avg_everytwo_water[n])
    cc_data.append(d)
    cc_wref.append(w)

In [ ]:
spectra = cc_data[1].spectrum()
frequency_slice = spectra.slice_ppm(3.5, 1.9)
plt.imshow(spectra[:, frequency_slice].T.real, extent=[0, 128, 1.9, 3.5], aspect='auto')

In [ ]:
spectra = cc_data[0].spectrum()
frequency_slice = spectra.slice_ppm(3.5, 1.9)
plt.imshow(spectra[:, frequency_slice].T.real, extent=[0, 128, 1.9, 3.5], aspect='auto')

In [ ]:
first = cc_wref[1][1]
plt.plot(first.frequency_axis_ppm(), first.spectrum().real)
plt.xlim([5,0])
plt.axvline(4.7)

In [ ]:
corr = suspect.processing.frequency_correction.residual_water_alignment(first)

In [ ]:
plt.plot(first.frequency_axis_ppm(), first.adjust_frequency(0).spectrum().real)
plt.plot(first.frequency_axis_ppm(), first.adjust_frequency(-corr).spectrum().real)
plt.xlim([5,4])
plt.axvline(4.7)

In [ ]:
first = first.adjust_frequency(-corr)
corr

In [ ]:
w_freq_corr  = [suspect.processing.frequency_correction.correct_frequency_and_phase(i,first) for i in cc_wref]

In [ ]:
first = cc_data[1][15]
plt.plot(first.frequency_axis_ppm(), first.adjust_phase(-3.3).adjust_frequency(0).spectrum().real)
plt.xlim([5,0])
plt.axvline(4.7)

In [ ]:
first = first.adjust_phase(-3.3).adjust_frequency(0)

In [ ]:
## 1.4 Coregister to first

In [ ]:
cc_data[0].shape

In [ ]:
[i.shape for i in cc_data]

In [ ]:
freq_corr2 = [suspect.processing.frequency_correction.correct_frequency_and_phase(i,first) for i in cc_data]

In [ ]:
# Average

In [ ]:
w_avg = [np.mean(i,axis=0) for i in w_freq_corr]
averaged = [np.mean(i,axis=0) for i in freq_corr2]

In [ ]:
# ECC

In [ ]:
avg_ecc = list()
for n in range(len(averaged)):
    eddy_current = np.unwrap(np.angle(w_avg[n]))
    ec_smooth = suspect.processing.denoising.sliding_gaussian(eddy_current, 32)
    ecc = np.exp(-1j * ec_smooth)
    ec_data = averaged[n] * ecc
    avg_ecc.append(ec_data)

In [ ]:
for n,i in enumerate(averaged):
    plt.plot(i.frequency_axis_ppm(), i.spectrum().real, label=n)
plt.xlim([5,0])
#plt.ylim([-2e-4,5e-4])
plt.axvline(4.7)
plt.legend()

In [ ]:
for n,i in enumerate(avg_ecc[:]):
    plt.plot(i.frequency_axis_ppm(), i.adjust_frequency(2).spectrum().real, label=n)
plt.xlim([5,0])
plt.ylim([-200,500])
plt.axvline(2.04)
plt.axvline(0.94)
plt.axvline(3.94)
plt.legend()
plt.axhline(0)

In [ ]:
avg_ecc = [i.adjust_frequency(8) for i in avg_ecc]

In [ ]:
for n,i in enumerate(avg_ecc):
    plt.plot(i.frequency_axis_ppm(), i.spectrum().real, label=n)
plt.xlim([5,0])
plt.ylim([-200,500])
#plt.axvline(1.99)
plt.legend()
#plt.axhline(0)

In [ ]:
der_path = '/home/orco/data/MacroMols/MRS/MacroMols_3T/derivatives/preprocessed'

In [ ]:
# Save as raw and npz
for n, i in enumerate(averaged):
    l = mm_dats[n].split('/')[-1].split('_')
    filename = mm_dats[n].split('/')[-1].replace('.dat', '_avg.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out_raw = os.path.join(out_path, filename)
    out_npz = out_raw.replace('.raw','.npz')
    print(out_npz)
    np.savez(out_npz, mrsdata=i)
    print(out_raw)
    suspect.io.lcmodel.save_raw(out_raw, i)

In [ ]:
# Save as raw ecc
for n, i in enumerate(avg_ecc):
    l = mm_dats[n].split('/')[-1].split('_')
    filename = mm_dats[n].split('/')[-1].replace('.dat', '_avg_ecc.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out_raw = os.path.join(out_path, filename)
    out_npz = out_raw.replace('.raw','.npz')
    print(out_npz)
    np.savez(out_npz, mrsdata=i)
    print(out_raw)
    suspect.io.lcmodel.save_raw(out_raw, i)

In [ ]:
# Save as water raw and npz
for n, i in enumerate(w_avg):
    l = water[n].split('/')[-1].split('_')
    filename = water[n].split('/')[-1].replace('.dat', '_avg.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out_raw = os.path.join(out_path, filename)
    out_npz = out_raw.replace('.raw','.npz')
    print(out_npz)
    np.savez(out_npz, mrsdata=i)
    print(out_raw)
    suspect.io.lcmodel.save_raw(out_raw, i)

In [ ]:
# Averaged MM Spectrum
avg_all = np.array(avg_ecc).mean(axis=0)
avg_all = avg_ecc[0].inherit(avg_all)
out_file = '/home/orco/data/MacroMols/MRS/MacroMols_3T/derivatives/LCModel_averaged/MM_avg_ecc_all.raw'
os.makedirs(os.path.dirname(out_file), exist_ok=True)
suspect.io.lcmodel.save_raw(out_file,avg_all)

In [ ]:
# HLSVD

In [ ]:
water_list = list()
dry_list = list()
for spec in avg_ecc:
    components = suspect.processing.water_suppression.hsvd(spec, 30)
    water_components = [component for component in components if component["frequency"] < 60]
    water_fid = spec.inherit(suspect.processing.water_suppression.construct_fid(water_components, spec.time_axis()))
    dry_fid = spec - water_fid
    water_list.append(water_fid)
    dry_list.append(dry_fid)

In [ ]:
for i in water_list:
    plt.plot(i.frequency_axis_ppm(), 
             i.spectrum().real)
    plt.xlim([5,0])
    plt.ylim([-200,500])
    #plt.axvline(4.7)

In [ ]:
for i in dry_list:
    plt.plot(i.frequency_axis_ppm(), 
             i.adjust_frequency(-3).spectrum().real)
    plt.xlim([5,0])
    plt.ylim([-200,500])
    plt.axvline(4.2)
    plt.axvline(0.94)
    plt.axvline(2.04)

In [ ]:
dry_list = [i.adjust_frequency(-3) for i in dry_list]

In [ ]:
# Save as raw ecc
for n, i in enumerate(dry_list):
    l = mm_dats[n].split('/')[-1].split('_')
    filename = mm_dats[n].split('/')[-1].replace('.dat', '_avg_ecc_dry.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out_raw = os.path.join(out_path, filename)
    out_npz = out_raw.replace('.raw','.npz')
    print(out_npz)
    np.savez(out_npz, mrsdata=i)
    print(out_raw)
    suspect.io.lcmodel.save_raw(out_raw, i)

In [ ]:
# Averaged MM Spectrum
avg_all = np.array(dry_list).mean(axis=0)
avg_all = avg_ecc[0].inherit(avg_all)
out_file = '/home/orco/data/MacroMols/MRS/MacroMols_3T/derivatives/LCModel_averaged/MM_avg_ecc_dry_all.raw'
os.makedirs(os.path.dirname(out_file), exist_ok=True)
suspect.io.lcmodel.save_raw(out_file,avg_all)